In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

df = pd.read_csv(r"DatosVino.csv", sep=",")
#display(df)
#La columna clasificación tiene valores raros
df["Clasificacion"].unique()
#ver si el valor "?" es significativo. Lo es, no debería borrarse
df["Clasificacion"].value_counts()
#convertimos sus valores a numéricos para entender mejor los datos
df["Clasificacion"] = df["Clasificacion"].apply(lambda x: 0 if x == "?" 
                                                else(1 if x=="*"
                                                else(2 if x=="**" else(3 if x=="***" else(4)))))
df["Clasificacion"].value_counts()
#Observamos los null del dataset
df.isnull().sum()
#Nos centramos en la columna Region. El % de nulos es bastante bajo, SE BORRAN
df.dropna(subset = ["Region"], inplace = True)
df.isnull().sum()
#Nos centramos en la columna Sulfatos. 
df_copy = df.copy()
df["Sulfatos"] = df["Sulfatos"].fillna(df["Sulfatos"].mean())
df
df.isnull().sum()
#Hacemos un .info para observar el tipo de datos que tenemos. Puede que haya algunos tipos por cambiar y ahorrar recursos de la máquina
#Por ejemplo, la columna Region
df["Region"] = df["Region"].astype("int8")
#df_copy.info()

#Agrupar los datos de beneficio
df["Beneficio"].value_counts(100) #al poner el 100 da el % que representan los datos

#Hacemos un boxplot para observar la distribución de los datos
#sns.boxplot(x=df['Beneficio'])
#plt.title('Distribución Beneficio')
#plt.show()
#Hacemos un histograma para observar la distribución de los datos
#sns.histplot(x=df['Beneficio'])
#plt.title('Distribución Beneficio')
#plt.show()
#Hay muchos datos en el valor 0 y todos los demás valores no son representativos. Vamos a normalizar los datos entre el 0 y el 1 con un minmaxscaler
scaler = MinMaxScaler(feature_range = (0,1))
df["Beneficio"] = scaler.fit_transform(df[["Beneficio"]])

#Observamos la columna Etiqueta
df["Etiqueta"].unique()
#convertimos los valores a minusculas
df['Etiqueta'] = df['Etiqueta'].str.upper()
df["Etiqueta"].unique()
#Convertimos a numérico
# Definir el orden de las categorías
orden_categorias = ['MM', 'M', 'R', 'B', 'MB']
 
# Crear un diccionario de mapeo de categorías a números
mapa_categorias = {categoria: indice for indice, categoria in enumerate(orden_categorias)}
 
# Reemplazar los valores de la columna 'A' usando el diccionario de mapeo
df['Etiqueta'] = df['Etiqueta'].map(mapa_categorias)
df["Etiqueta"]

#Observamos la columna azucar que se supone que tienen que ser todos positivos
df["Azucar"].min()
#Observamos la columna Densidad que se supone que tienen que ser todos positivos
df["Densidad"].min()
#Observamos la columna Alcohol que tiene que tener valores entre 1 y 100
df["Alcohol"].max()
#Hay que tratar la columna Alcohol para que los valores estén correctos. Convertimos los valores incorrectos a nulos
df['Alcohol'] = df['Alcohol'].apply(lambda x: None if x > 100 else x)
df['Alcohol'] = df['Alcohol'].apply(lambda x: None if x < 0 else x)
df['Clasificacion'] = df['Clasificacion'].apply(lambda x: None if x == 0 else x)
df.isnull().sum()
#Tratamos los nulos
imputer = KNNImputer()
keyColums =['Beneficio', 'Compra', 'Acidez', 'AcidoCitrico', 'pH',
       'CloruroSodico', 'Densidad', 'Azucar', 'Sulfatos', 'Alcohol',
       'Etiqueta', 'CalifProductor', 'Clasificacion', 'Region',
       'PrecioBotella']
df[keyColums]=imputer.fit_transform(df[keyColums])
df.isnull().sum()

df.duplicated().sum()
#Entrenamos el modelo. Dividimos el dataset en test y train.
X = df.iloc[:,:-1]
y = df["PrecioBotella"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

kfold = KFold(n_splits=10)
model=LinearRegression()
result = cross_val_score(model, X, y, cv=kfold)
result



array([-0.01323806, -0.0087448 ,  0.00041109, -0.00818066,  0.00015711,
       -0.00470759, -0.00388188,  0.00297765, -0.00683823, -0.00445125])